In [179]:
from mat_dataset import *
import numpy as np

import torch
import torch.nn as nn
from math import pow    
# import fastai

# ----------------------------------------------------------------------------------
# Functions to handle the unique attributes of the consensus1.mat file in Gong Lab
# @author: davidebear
# 
# Note: Refer to https://github.com/emiliosalazar/matToPython/blob/main/MatFileMethods.py for a more encompassing MatLabLoader
# ----------------------------------------------------------------------------------

# Needed imports
import numpy as np
import h5py

def load_matlab_file(mat_file, variable_name): # Note the double return statement
    """_summary_
    //UNIQUE: Only one variable for this .mat file
    
    Args:
        mat_file (string): Name of .mat file to intake (path must be given if not in current dir)
        variable_name (string): which variable from the mat file. Only one here because of the .mat file structure.

    Returns:
        h5py_object, data: Overall h5py_object and the specific variable as an nparray
    """
    h5py_object = h5py.File(mat_file, 'r')
    data = h5py_object.get(variable_name)
    data = np.array(data)
    return h5py_object, data

class DNA_SeqBlocks():
    """_summary_
    Loads the mat file with DNA sequences for easy iteration through all stored DNA sequence lists of varying coverage
    """
    def __init__(self, h5py_object, data):
        """_summary_
        Args:
            h5py_object (h5py_object): first return variable of load_matlab_file
            data (np.array): second return variable of load_matlab_file
        """
        self.h5py_object = h5py_object
        self.data = np.transpose(data)
        self.size = len(data) 
    def get_seqblock(self, number): # //TODO: make sure the default args work
        x = self.h5py_object[self.data[0, number-1]][:, :] # //UNIQUE: 0th column because .mat file, align3 variable only has one row
        return np.transpose(x)
    
# CURRENT ISSUE:  I'm getting an int array instead of a char array. Use pd dataframe? How do I force a char?
def seqblock_parser(seqblock):
    seqblock_parsed = CleanSeqBlock()
    total_columns = len(np.transpose(seqblock)) # is there a more efficient way?
    total_rows = len(seqblock)
    number_of_reads = int((total_rows - 4)/2) # last three rows have intreptations, first row is non-mutated target sequence, and N quality scores for N reads
    
    seqblock_parsed.reads_count = number_of_reads;
    seqblock_parsed.seq_len = total_columns;
    
    # print(total_columns)
    # print(total_rows)
    
    for i in range(0,total_rows-1):
        # print(f" The sequenceblock input {seqblock}") #debug
        # print(f" The m x n size of the seqblock {seqblock.shape}") #debug
        # print(f" The ith row of the seqblock {current_seq}") #debug
        # print(f" The m x n size of the sequence {current_seq.shape}") #debug
        new_format = np.chararray(total_columns)
        new_format[:] = 'q' #debug
        
        # print(f" The 0th entry of the initialized char array {new_format[0, 0]}") #debug
        # print(f" Its row size {len(new_format[0])}")
        for j in range(total_columns):
            # print(j)
            new_format[j] = chr(seqblock[i,j])
            
            
        if (i == 0):
            seqblock_parsed.target = new_format
        elif (i == total_rows - 1): # last row is mutations 'x', total_rows is one more than total index
            seqblock_parsed.interp_mutations = new_format
        elif (i == total_rows - 2): # 2nd last is subjective consensus
            seqblock_parsed.interp_consensus = new_format
        elif (i == total_rows - 3): 
            seqblock_parsed.interp_changes = new_format
        elif (i > 0 and i < number_of_reads + 1):
            seqblock_parsed.reads.append(new_format)
        else:
            seqblock_parsed.qscores.append(new_format)
    return seqblock_parsed     
    
class CleanSeqBlock():
    def __init__(self):
        self.reads_count = 0;
        self.seq_len = 0;
        
        self.target = 0;
        
        self.reads = [];
        self.qscores = [];
        
        self.interp_changes = 0; # indel in between y's and z shows a KNOWN base error
        self.interp_consensus = 0;
        self.interp_mutations = 0;
    


In [201]:
# Load the mat file in
file_name = 'consensus1.mat'
variable_of_interest = 'align3'
h5py_object, data = load_matlab_file(file_name, variable_name=variable_of_interest)

DNA_seqs = DNA_SeqBlocks(h5py_object=h5py_object, data=data)

In [181]:
# TEST FOR SEQBLOCK_PARSER
test_1 = DNA_seqs.get_seqblock(1) # Let's look at the first DNA sequence block
test_parse = seqblock_parser(test_1) # Let's test the parsing function

In [182]:
test_quality = test_parse.qscores[0] #Let's look at the first quality strand
test_read = test_parse.reads[2] # Let's look at the first read strand

In [183]:
#Read strand Testing for info loss


print(f"Loss Checker for {test_parse.reads_count} reads:")
for j in range(test_parse.reads_count):
    Ccount = 0
    Tcount = 0
    Acount = 0
    Gcount = 0
    indelcount = 0
    for i in test_parse.reads[j]: #Input change here
        if i == b'C':
            Ccount+=1;
        if i == b'A':
            Acount+=1;
        if i == b'T':
            Tcount +=1;
        if i == b'G':
            Gcount +=1;
        if i == b'-':
            indelcount +=1;
    # print(f" C: {Ccount} \n G: {Gcount} \n A: {Acount} \n T: {Tcount} \n indels: {indelcount}")
    # print(test_parse.seq_len)
    assert Ccount+Tcount+Gcount+Acount+indelcount == test_parse.seq_len, f"Info lost in {j}th index read "
    print(f"Success in {j}th read!")

Loss Checker for 3 reads:
Success in 0th read!
Success in 1th read!
Success in 2th read!


In [199]:
# Quality Strand Testing
#//TODO: Can I convert each char into a quality score value?

def error_calculator(list_qscores):
    for i in range(len(list_qscores)): #often more than one
        for n in list_qscores[i]:
            if (n == b'~'): continue
            q = ord(n)-64
            P = pow(10, -q/10)
            print(P)
            
        
error_calculator(test_parse.qscores)

0.15848931924611134
1.584893192461114e-06
1258.9254117941675
0.07943282347242814
7.943282347242822e-05
63.09573444801933
1.9952623149688787e-06
5.011872336272725e-06
2.5118864315095822e-05
7.943282347242822e-07
0.012589254117941675
7.943282347242822e-07
0.025118864315095794
0.000630957344480193
0.5011872336272722
